In [92]:
import pandas as pd
import numpy as np

# Read the CSV file
stock_data = pd.read_csv("../Data/all_stocks_5yr_clean.csv")

# Getting the unique ticker names
tickers = stock_data['Name'].unique()

# Creating a dictionary of data frames with ticker names as keys
ticker_dfs = {ticker: stock_data[stock_data['Name'] == ticker] for ticker in tickers}

# Calculate average return, compounded return and standard deviation of each stock
avg_returns = []
std_returns = []
compounded_returns = []
raw_returns = [] #the unaggregated data is required for calculating covariance during optimization

for ticker, ticker_df in ticker_dfs.items():
    # Convert 'Date' to datetime
    ticker_df['date'] = pd.to_datetime(ticker_df['date'])
    
    # Set 'Date' as the index
    ticker_df.set_index('date', inplace=True)
    
    # Resample the data to find the last date of each month
    ticker_df_new = ticker_df.resample('M').last()
    ticker_df_new = pd.concat([ticker_df.iloc[:1], ticker_df_new])

    # Calculate the monthly returns
    ticker_df_new['monthly_return'] = ticker_df_new['close'].pct_change()
    
    # Drop all NaN values
    ticker_df_new = ticker_df_new.dropna()
    
    # Calculate the compounded returns, average returns and standard deviation
    avg_return = ticker_df_new['monthly_return'].mean()
    std_return = ticker_df_new['monthly_return'].std()
    compounded_return = (1 + ticker_df_new['monthly_return']).cumprod()[-1] - 1
    
    # Append the returns to the global lists
    avg_returns.append(avg_return)
    std_returns.append(std_return)
    compounded_returns.append(compounded_return)
    raw_returns.append(ticker_df_new['monthly_return'])

# Create new data frame for the extracted returns
data = {'Ticker': tickers, 'Average Return': avg_returns, 'Compounded Return': compounded_returns, 'Standard Deviation': std_returns}
raw_data = {'Ticker': tickers, 'Returns': raw_returns}
stock_new_df = pd.DataFrame(data)
#Create a covariance matrix of the mean returns; necessary for calculating stock volatility for Sharpe ratio
raw_returns_df = pd.DataFrame(index=tickers, data=raw_returns)
covariance_df = pd.DataFrame(data=np.cov(np.array(raw_returns_df), rowvar=True), index=tickers, columns=tickers)

# Export the data frame and covariance df as csv file
stock_new_df.to_csv('../Data/SP500_compounded_return.csv', index=False)
covariance_df.to_csv('../Data/SP500_mean_return_covariance.csv', index=False)

C:\Users\danie\AppData\Local\Temp\ipykernel_17560\1859408461.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df['date'] = pd.to_datetime(ticker_df['date'])
C:\Users\danie\AppData\Local\Temp\ipykernel_17560\1859408461.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ticker_df['date'] = pd.to_datetime(ticker_df['date'])
C:\Users\danie\AppData\Local\Temp\ipykernel_17560\1859408461.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r